In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.ned import Ned
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from urllib.request import urlretrieve
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from astropy.io import fits
from astropy.table import Table
import glob
import os
from scipy.interpolate import CubicSpline
import textwrap
import seaborn as sns
import scipy.stats as scp


def make_desig(data, ra_key='ra', dec_key='dec'):
    """make designation if df has 'ra' and 'dec' columns"""
    desig=[]
    for posstring in SkyCoord(data[ra_key].values*u.deg, data[dec_key].values*u.deg).to_string("hmsdms"):
        posstring = posstring.split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        desig.append('J'+des_ra+des_dec)
    return desig

def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def cross_bigmac(mul171):
    """crossmatch sample with big MAC"""
    # read in big  mac
    bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
    # format designation
    desigs = []
    for i in range(len(bigmac)):
        name = bigmac['Name1'].loc[i].replace("SDSS","")
        if name[0] == "J":
            if "+" in name:
                desig = name.split("+")[0][:5] + "+" + name.split("+")[1][:4]
                desigs.append(desig)
            elif "-" in name:
                desig = name.split("-")[0][:5] + "-" + name.split("-")[1][:4]
                desigs.append(desig)
            else: print(name) 
        else:
            desigs.append(name)
    bigmac['DESIG'] = desigs

    # merge big mac and mullaney
    mul_bm = pd.merge(mul171,bigmac, on="DESIG")

    # optionally can get decals images
    # for n in mul_bm.index:
    #     urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra='+str(mul_bm.loc[n,'RA'])+'&dec='+str(mul_bm.loc[n,'DEC'])+'&layer=decals-dr7&pixscale=0.27&bands=grz',
    #                 "/home/insepien/research-data/hst/mul_bm/"+str(mul_bm.loc[n,'DESIG'])+'.jpg')
    return mul_bm, bigmac

def cal_sep(theta, z):
    """return dual sep in kpc given scalar angle sep in arcsec"""
    angle = (theta*u.arcsec).to(u.rad).value
    return (cosmo.angular_diameter_distance(z)*angle).to(u.kpc)


def f(on, theta):
    """plot decal image and annulus at detected dual separation theta"""
    fn = "/home/insepien/research-data/hst/mul_bm/"+on+".jpg"
    decals_plate_scale = 0.236 #''/pix
    pix_sep = theta/decals_plate_scale

    fig,ax = plt.subplots()
    im = plt.imread(fn)
    midF = im.shape[0]/2
    ax.imshow(im)
    circ = plt.Circle((midF,midF), pix_sep, fill=False, color='white',alpha=0.5,label=f"{theta:.2f}''")
    ax.add_patch(circ)
    ax.legend()
    ax.set_title(on)


### check emission line luminosities
J1010 is dimmer than rest of sample based on OIII dereddened, but is brighter than the rest in SDSS R-band. R-band covers OIII and HB, so it should be OIII dominated. Therefore OIII dereddened measurments do not make sense

In [ ]:
#### check emission lines
df = Table(fits.getdata("/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits")).to_pandas()
j10 = df[df['Desig'] == "J1010+1413"]
with open("/home/insepien/research-data/alpaka/alpaka_39fits.pkl","rb") as f:
    magel = pickle.load(f)
keys = ['HB_LUM',
 'OIII_4959_LUM',
 'OIII_5007_LUM',
 'NII_6548_LUM',
 'HA_LUM',
 'NII_6584_LUM',
 'NVS_LUM']
i=0
clr = sns.color_palette("colorblind", len(keys))
for k in keys:
    lums = [m for m in magel[k] if np.isfinite(m) and m!=0]
    plt.hist(np.log10(lums),color=clr[i],alpha=0.7,label=k)
    try:
        plt.axvline(np.log10(j10[k].values),c=clr[i])
    except:
        print(k)
    i+=1
plt.xlim((37.5,43.5))
plt.legend(bbox_to_anchor=(1,1));

### cross-match catalogs
look at the 152 sample when doing HST proposal and cross-match with Big MAC. Also plot dual candidates; this could be used for intro?

In [ ]:
# read in original sample
mul171 = pd.read_pickle("/home/insepien/research-data/alpaka/mull152.pkl")
mul171['DESIG'] = make_desig(mul171, ra_key='RA',dec_key="DEC")
mul_bm, bigmac = cross_bigmac(mul171)

# get some stats on sub-kpc pairs for science justification
subkpc_mask = ((bigmac['Primary System Type']=='Dual AGN Candidate') | (bigmac['Primary System Type']=='Dual AGN')) & (bigmac['Sep(kpc)']<1) & (bigmac['Sep(kpc)']>0)
subkpc_dual = bigmac[subkpc_mask]
dual_mask = ((bigmac['Primary System Type']=='Dual AGN Candidate') | (bigmac['Primary System Type']=='Dual AGN')) & (bigmac['Sep(kpc)']>1)
dual = bigmac[dual_mask]
print("fraction of sub-kpc", len(subkpc_dual)/len(dual))
print("number of duals", len(dual))

# get some methods of measuring sub-kpc sep
anyl_meth = Counter(subkpc_dual['Parsed Analysis Method'])
print("1st most common method: ",anyl_meth.most_common(2)[0])
print("2nd most common method: ",anyl_meth.most_common(2)[1])


# cross-match specifically for magellan
with open("/home/insepien/research-data/alpaka/alpaka_39fits.pkl","rb") as f:
    magel = pickle.load(f)

magel.reset_index(inplace=True)
magel.rename(columns={"Desig":"DESIG"},inplace=True)
magel_bm, _ = cross_bigmac(magel)

In [ ]:
from matplotlib.lines import Line2D
import matplotlib.patches as patches
sns.set_context("paper",font_scale=1.75)
sns.set_style('ticks')
sns.set_palette('colorblind')
figparams = {'font.family': 'DejaVu Sans',
            'font.serif':'Times',
            'hatch.linewidth' : 3.0}
plt.rcParams.update(figparams)

fig,ax = plt.subplots(figsize=(15,7),dpi=500)
# plot hst resolution limit
for reso in [1,5,20]:
    seeing = 0.04*2.5*reso
    ax.plot(np.linspace(0,3),cal_sep(seeing, np.linspace(0,3)),c='k',linestyle="--",alpha=0.5)
    ax.text(2.5, 1*reso, f"{seeing:.1f} arcsec", color='k',fontsize=15)

# add survey vol
# rect = patches.Rectangle((0.1, 0), 0.4-0.1, 1, linewidth=2, edgecolor='w', facecolor='darkseagreen',alpha=0.3)
# ax.add_patch(rect)
# ax.text(0.2,1,"Our survey",c="darkolivegreen",fontsize=15)

# kpc points ie sep>1 with confidence>0.5, z0-3, have been imaged
conf_kpc = dual['ST1 Confidence Flag']>=0.5
imag_kpc = dual['Parsed Analysis Method'].str.contains("Imaging")
z_mask = (dual['z1']>0)&(dual['z1']<3)
kpcdf = dual[conf_kpc&imag_kpc&z_mask]
paper = list(kpcdf['Paper(s)'].str.split(" ; ").explode().value_counts().keys())[:15]
all_markers = list(Line2D.markers.keys())[5:]
all_colors = sns.color_palette("colorblind", len(paper))
for j in range(len(paper)):    
    papermask = np.array([np.isin(paper[j],kpcdf.loc[i,'Paper(s)'].split(" ; ")).item() for i in kpcdf.index.values])
    for k in kpcdf[papermask].index.values:
        ulabel = paper[j] if k==kpcdf[papermask].index.values[0] else None
        plt.scatter(kpcdf[papermask]['z1'][k],kpcdf[papermask]['Sep(kpc)'][k],marker=all_markers[j],c=all_colors[j],s=30,label=ulabel)

# plot points with confidence >0.5 and have been imaged
confidence_mask = subkpc_dual['ST1 Confidence Flag']>=0.5
imaging_mask  = subkpc_dual['Parsed Analysis Method'].str.contains("Imaging")
distance_mask = subkpc_dual['Sep(kpc)']>0.2  # to remove radio sources
df = subkpc_dual[confidence_mask&imaging_mask&distance_mask]
lagn = np.log10([0.033e44,0.135e44,3e44,10**43.23*600,6e46])
subkpc_markers = list(Line2D.markers.keys())[1:6]
for i,m,mrk in zip(df.index.to_list(),lagn,subkpc_markers): # mark confirmed and very sure candidates differently
    wrapped_label = "\n".join(textwrap.wrap(df['Paper(s)'][i], width=60))
    sca = ax.scatter(df['z1'][i], df['Sep(kpc)'][i],label=wrapped_label,s=30,cmap='magma',c=m,marker=mrk,vmin=np.min(lagn)-0.5,vmax=np.max(lagn)+1)

cbar_ax = fig.add_axes([0.99, 0.125, 0.01, 0.75]) 
cbar = fig.colorbar(sca,cax=cbar_ax)
cbar.set_label("Log($L_{AGN}$) [erg/s]")

ax.set_xlabel("Redshift")
ax.set_ylabel("Projected separation [kpc]")
ax.set_xlim((-0.01,3))
ax.set_ylim((0.2,110))
# set top xlabel to look back time
ax_top = ax.secondary_xaxis("top")
ax_top.set_xlabel("Lookback time [Gyr]")
# interpolate to get tick positions for round lookback time
spl = CubicSpline(cosmo.lookback_time(np.linspace(0,0.5)),np.linspace(0,0.5))
xtick_pos = spl(np.arange(1,12,2))
ax_top.set_xticks(xtick_pos)
ax_top.set_xticklabels(np.arange(1,12,2))
ax.set_yscale('log')

ax.legend(ncol=3,fontsize=8,loc='lower center')
ax.grid(linestyle='--',alpha=0.5)
fig.tight_layout()
fig.savefig("hst.png",dpi=500);

### Dual params
get luminosity of Magellan sample

In [ ]:
import astropy.constants as const
def get_wise_mags(wise_):
    """get wise mags and mag errors from data frame of ipac search results"""
    ## get wise mags and errors
    w1mag = wise_['w1mpro']
    w2mag = wise_['w2mpro']
    w3mag = wise_['w3mpro']
    w4mag = wise_['w4mpro']
    wmags_ = np.array([w1mag, w2mag, w3mag, w4mag])
    wmags_err_ = np.array([wise_['w1sigmpro'], wise_['w2sigmpro'], wise_['w3sigmpro'], wise_['w4sigmpro']])
    return wmags_, wmags_err_


def wise_lum_from_mag(wmags_, wmags_err_, obs_wavelength_, redshift_):
    """calculate wise luminosity from magnitude at some observed wavelength"""
    ## change mags to fluxes -- http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html#example
    zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
    fluxdens = zeromagflux*10**(-wmags_/2.5) # in Jy
    # now either interpolate flux dens to some wavelength or use a band from wise
    wise_wavelengths = np.array([3.4, 4.6, 12., 22.]) # 1e-6 m
    if np.isin(obs_wavelength_, wise_wavelengths): # check if need to interpolate to non-wise wl
        obs_flux = fluxdens[wise_wavelengths==obs_wavelength_][0]
    else: # interpolate
        fluxdens_err = zeromagflux*10**(-wmags_err_/2.5)
        ## interpolate - use straight line
        wiseflux = np.polyfit(np.array(wise_wavelengths)[[0,1,3]], np.array(fluxdens.value)[[0,1,3]],deg=1, w=1./np.array(fluxdens_err)[[0,1,3]])
        ## get flux at obs wavelength, i.e. just a straight line here
        obs_flux = (wiseflux[0]*obs_wavelength_+wiseflux[1])*u.Jy    
        #plt.scatter(wise_wavelengths, fluxdens.value)  
        #plt.plot(wise_wavelengths, wise_wavelengths*wiseflux[0]+wiseflux[1])
    ## change to luminosity
    obs_hz = (const.c/(obs_wavelength_*u.micron)).to(u.Hz)
    lum = (obs_flux*obs_hz*4*np.pi*
           cosmo.luminosity_distance(redshift_)**2).to(u.erg/u.s)
    return lum


def correct_ir():
    """correct IR luminosity at 15 microns rest frame based on Hopkins+20"""
    # load hopkins bolometric correction
    with open("/home/insepien/research-data/pop-result/bc.txt","r") as f:
        d = f.read().splitlines()
    hopkins = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
    Lbol = np.array(list(hopkins['Lbols'].values), dtype=float)
    LIR = np.array(list(hopkins['LIRs'].values), dtype=float)
    spl = CubicSpline(LIR, Lbol)
    return spl

def get_wise_ir_lums(wise_, alpaka_,wise_key = 'designation', mul_key='Desig',wl_=22):
    """calculate wise IR luminosity and bolometric lum, 
        default keys are for magellan sample"""
    ## get wise mags and errors
    wmags, wmags_err_nan = get_wise_mags(wise_)
    # replace nan values in mag error with median
    wmags_err = np.nan_to_num(wmags_err_nan,np.median(wmags_err_nan))
    # calculate luminosity
    wise_lums = np.zeros((len(wise_)))
    for i in range(0, len(wise_)):
        on = wise_.loc[i,wise_key]
        z = np.mean(alpaka_[alpaka_[mul_key]==on]['Z'])
        wise_lums[i] = wise_lum_from_mag(wmags[:,i], wmags_err[:,i], wl_, z).value
    # check wavelength to see how to do bolo correction
    if wl_==15: # use Hopkins+2020 if at 15 microns
        spl = correct_ir()
        irbol = 10**(spl(np.log10(wise_lums)))
    else: # else correct by 12%
        irbol = wise_lums*10**1.12
    return wmags, wise_lums,irbol

def crop_cat(namelist,wsearch,mul):
    wmask = wsearch['desig'].isin(namelist)
    wcat = wsearch[wmask].reset_index()
    mmask = mul['desig'].isin(namelist)
    mcat = mul[mmask].reset_index()
    return wcat, mcat

def lbol_to_m(lbol,edd_rate=0.3):
    ledd = lbol/edd_rate
    return np.log10(ledd/(1.28e46/1e8))

In [ ]:
# load wise match of all type 2 agn in mullaney
wsearch =  pd.read_pickle("/home/insepien/research-data/alpaka/wise-cat/all_type2_wise.pkl")
wsearch['desig'] = make_desig(wsearch)
# load alpaka and match with wise, for sample plotting later
alpaka = Table(fits.open("/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits")[1].data).to_pandas()
wise_names = wsearch['desig']
alpaka_in_wise = alpaka[alpaka['Desig'].isin(wise_names)]
alpaka_in_wise.rename(columns={'Desig':'desig'},inplace=True)
maindf = pd.merge(alpaka_in_wise, wsearch,on='desig',how='left')
# load magellan 39 sample
mul = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
mul['desig'] = make_desig(mul, ra_key='RA',dec_key="DEC")
# crop single and dual magellan subsets
dualnames = ["J1215+1344","J1222-0007"]
wdual, mdual = crop_cat(dualnames,wsearch,mul)
singleAGNnames =  mul[~ mul['desig'].isin(dualnames)]['desig']
wsingle, msingle = crop_cat(singleAGNnames,wsearch,mul)
# calculate Lbol from WISE and BH mass
wmags_dual, wise_lums_dual,irbol_dual = get_wise_ir_lums(wdual,mdual,wise_key="desig",mul_key='desig',wl_=15)
wmags, wise_lums,irbol = get_wise_ir_lums(wsingle, msingle,wise_key="desig",mul_key='desig',wl_=15)
print(wise_lums_dual)
print(irbol_dual)
print("BH mass ", lbol_to_m(irbol_dual))

### some sample plots
first IR only

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})
sns.set_context("paper",font_scale=1.75)


fig,ax = plt.subplots(2,2,gridspec_kw={'width_ratios': [2,0.5],'height_ratios': [0.7,2]},figsize=(8,6),
                      sharey='row',sharex='col',dpi=300)
plt.subplots_adjust(wspace=0.05,hspace=0.05)

mdual_labs = ['J1215+1344','J1222-0007 E', 'J1222-0007 W']
ax[1,0].scatter(maindf['Z'],np.log10(maindf['irbol']),s=2,alpha=0.1,color="plum")
ax[1,0].scatter(msingle['Z'],np.log10(irbol),s=50,marker='2',color='indigo',alpha=0.7)
[ax[1,0].scatter(mdual['Z'][:2][i],np.log10(irbol_dual)[i],s=40,marker=['s','o'][i],color='indigo',alpha=0.7,label=mdual_labs[:2][i]) for i in range(2)];

ax[1,0].set_ylim(43.5,47.5)
ax[1,0].set_xlabel("Redshift")
ax[1,0].set_ylabel('Log($L_{bol,IR}$) $[erg s^-1]$')
ax[1,0].legend(fontsize=10) 

def norm_hist(ax,quant,ecolor,fcolor,bin_arr=[],alpha=1,horz=False):
    """normalize histogram sum count to 1 given some quantity (quant)
        args: edgecoloe, facecolor, bin array, opacity, flag for plotting horizontal hist"""
    count, bin = np.histogram(quant,bins=bin_arr)
    if horz:
        ax.barh(bin[:-1],count/np.sum(count), height= np.diff(bin),align='edge',edgecolor=ecolor,facecolor=fcolor,alpha=alpha)
    else:
        ax.bar(bin[:-1],count/np.sum(count),width = np.diff(bin),align='edge',edgecolor=ecolor,facecolor=fcolor,alpha=alpha)

binz = np.linspace(np.min(maindf['Z']),np.max(maindf['Z']),20)
norm_hist(ax[0,0],maindf['Z'],"plum",'none',bin_arr=binz)
norm_hist(ax[0,0],np.concatenate([msingle['Z'],mdual['Z']]),'none','indigo',bin_arr=binz,alpha=0.5)
ax[0,0].set_ylabel("f$_{AGN}$")

binL = np.linspace(np.log10(np.min(maindf['irbol'])), np.log10(np.max(maindf['irbol'])),10)
norm_hist(ax[1,1],np.log10(maindf['irbol'].dropna()),'plum',"none",binL,horz=True)
norm_hist(ax[1,1],np.log10(np.concatenate([irbol,irbol_dual])),'none','indigo',binL,alpha=0.5,horz=True)
ax_top = ax[1,1].secondary_xaxis("top")
ax_top.set_xlabel("f$_{AGN}$")
ax_top.set_xticks([0,0.2])
ax_top.set_xticklabels([0,0.2])
ax[1,1].set_xticks([])

ax[0,1].axis('off')

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})
sns.set_context("paper",font_scale=1.75)

fig, ax = plt.subplots(1,2,figsize=(12,4),sharey=True,dpi=500)
ax[0].scatter(maindf['Z'],np.log10(maindf['OIII_5007_LUM_DERRED']*800),s=2,alpha=0.1,color='plum')
ax[0].scatter(mul['Z'],np.log10(mul['OIII_5007_LUM_DERRED']*800),s=50,alpha=0.5,marker="2",color='indigo')
mdual_labs = ['J1215+1344','J1222-0007 W', 'J1222-0007 E']
[ax[0].scatter(mdual['Z'][i],np.log10(mdual['OIII_5007_LUM_DERRED']*800)[i],s=40,marker=['s','o',"^"][i],color='indigo',alpha=0.5,label=mdual_labs[i]) for i in range(3)];

ax[1].scatter(maindf['Z'],np.log10(maindf['irbol']),s=2,alpha=0.1,color='plum')
ax[1].scatter(msingle['Z'],np.log10(irbol),s=50,marker='2',color='indigo',alpha=0.5)
[ax[1].scatter(mdual['Z'][:2][i],np.log10(irbol_dual)[i],s=40,marker=['s','^'][i],color='indigo',alpha=0.5,label=[mdual_labs[0],mdual_labs[2]][i]) for i in range(2)];

ax[0].set_ylim(43,48)
[a.set_xlabel("Redshift") for a in ax]
[ax[i].set_ylabel(['Log($L_{bol,[OIII] dered}$) $[erg s^-1]$','Log($L_{bol,IR}$) $[erg s^-1]$'][i]) for i in range(2)]
[ax[i].legend(fontsize=10) for i in range(2)]
;

# try resampling to correct incomplete lum

In [ ]:
# get all type 2 agn in z cut from mul with match in wise
type2 = maindf[(maindf['Z'] > 0.14) & (maindf['Z'] < 0.22) & (maindf['AGN_TYPE']==2)]
magel_withwise = type2[type2['desig'].isin(magel['desig'])]
# get histograms, using sqrt smaller sample as numbers of bin
lbol_all = np.log10(type2['irbol'])
bin = np.linspace(lbol_all.min(),lbol_all.max(),int(np.ceil(np.sqrt(39))))
hist_ful = plt.hist(lbol_all,label="mullaney type-2 agn \nmatched with WISE",bins=bin)
hist_magel = plt.hist(np.log10(np.concatenate([irbol,irbol_dual])),label='Magellan sample',bins=bin)
plt.yscale('log')
plt.xlabel('Log(L_bol)')
plt.ylabel("number of AGN")
plt.legend();

In [ ]:
# interpolate pdf for pretty plotting
binmid = (bin[:-1]+bin[1:])*0.5
pdf_full = CubicSpline(binmid,hist_ful[0])
pdf_magel = CubicSpline(binmid,hist_magel[0])
# use full sample pdf as weight
w = hist_ful[0]/np.sum(hist_ful[0])
# assign weight to each magel target
binnum = np.digitize(np.log10(magel_withwise['irbol']),bin)-1
weights = [w[b] for b in binnum]
magel_withwise['weights'] = weights
# sample magel with weights
new_sample_size = 20
magel_sub = [magel_withwise.sample(n=new_sample_size,weights='weights') for i in range(1000)]
pdfs_magel = [np.histogram(np.log10(magel_sub[i]['irbol']),bins=bin)[0] for i in range(1000)]
# random sample from full sample
pdfs = [np.histogram(np.log10(type2.sample(n=new_sample_size)['irbol']),bins=bin)[0] for i in range(1000)]

In [ ]:
maxind = np.argmax(hist_magel[0])
magel_corrected = w/w[maxind]*hist_magel[0]
magel_corrected_spl = CubicSpline(binmid,magel_corrected)

In [ ]:
# for pretty plot
x = np.linspace(binmid.min(),binmid.max(),20)
# plot full sample
plt.plot(x,pdf_full(x),c='b')
plt.scatter(binmid,hist_ful[0],c='b',label='full sample')
# plot magellan sample
plt.scatter(binmid,hist_magel[0],label='magel',c='r')
plt.plot(x,pdf_magel(x),c='r')
# correct by largest bin
plt.scatter(binmid,magel_corrected,c="g",label='corrected')
plt.plot(x,magel_corrected_spl(x),c='g')
# plot subsamples
[plt.plot(binmid, pdfs_magel[i],c='r',alpha=0.01) for i in range(500)];
[plt.plot(binmid, pdfs[i],c='b',alpha=0.01) for i in range(500)];

plt.plot(binmid[maxind], magel_corrected[maxind],c="r",marker="*",markersize=20)
plt.ylim(bottom=0.5)
plt.yscale('log')
plt.xlabel('Log(L_bol)')
plt.ylabel("number of AGN")
plt.legend(bbox_to_anchor=(1,1));

### malmquist correction
tried volume limited sample but our sample is already very bright, so doesn't work

In [ ]:
from astropy.cosmology import WMAP9 as cosmo
mlim = 22.2 # r-band
sdss = pd.read_pickle("/home/insepien/research-data/alpaka/sdss-cat/sdss_rband_171.pkl")
sdss40 = sdss[sdss['DESIG'].isin(mul['desig'])]
sdss40['r-mag'] = 22.5 - 2.5 * np.log10(sdss40['spectroFlux_r'])
sdss40 = pd.merge(sdss40,mul.rename(columns={'desig':'DESIG'})[['Z','DESIG']],on='DESIG')
sdss40['M'] = sdss40['r-mag']-5*np.log10((cosmo.angular_diameter_distance(sdss40['Z'])*u.Mpc/(10*u.pc).to(u.Mpc)))

Mlim = -16.5
mask = sdss40['M'] < Mlim
dmax = (10**((mlim-Mlim)/5)*10*u.pc).to(u.Mpc)
from scipy.interpolate import CubicSpline
spl = CubicSpline(cosmo.angular_diameter_distance(np.linspace(0,1,20)), np.linspace(0,1,20))
zmax = spl(dmax.value)
zmax

### dual fraction from lit rev table

In [ ]:
dfr = pd.read_csv("/home/insepien/research-data/pop-result/lit_rev_frac/dualfrac_rev2.csv")
dfr_agn = dfr[~ dfr['lower dual frac'].isna()]
errs = []
not_nan_ind = dfr_agn['lower dual frac error'].dropna().index.values
nan_ind = dfr_agn.index[~ np.isin(dfr_agn.index,dfr_agn['lower dual frac error'].dropna().index.values)]
for i in range(len(dfr_agn)):
    if i in not_nan_ind:
        try:
            errs.append(np.abs(dfr_agn['lower dual frac'][i]-sorted(np.array(dfr_agn['lower dual frac error'][i].split(','),dtype=float))))
        except:
                print(i)
    else:
         errs.append(np.array([np.nan,np.nan]))
errs = pd.DataFrame(errs)

fmt = lambda minv, maxv: f"${minv:.0f}-{maxv:.0f}$"
fmt2 = lambda minv, maxv: f"${minv:.2f}-{maxv:.2f}$"
fmt_err = lambda val, low_err, up_err: f"${val:.3f}^{{-{low_err:.4f}}}_{{+{up_err:.4f}}}$"
fmt_lbol = lambda min,max : f"$10^{{{min:.0f}}}-10^{{{max:.0f}}}$"


redshift = [fmt2(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['min z'],dfr_agn['max z'])]
sep = [fmt(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['Min sep'],dfr_agn['Max sep'])]
lbols = [fmt_lbol(minvl,maxvl) for minvl,maxvl in zip(dfr_agn['min Lbol'],dfr_agn['max Lbol'])]
fracs = [fmt_err(val,errl,erru) for val,errl,erru in zip(dfr_agn['lower dual frac'],errs[0],errs[1])]

keys = ['Paper Name', 'Red shift', 'Separation $[kpc]$', "$L_{bol}~[erg~s^{-1}$]", "Selection method", "Dual fraction", "Fraction definition"]
tabb=pd.DataFrame([list(dfr_agn['Paper']),redshift,sep,lbols,list(dfr_agn['selection']),fracs,list(dfr_agn['note'])],index=keys).T.to_latex(column_format='c|c|c|c|c|c',index=False)
print(tabb)
